In [74]:
# Data
using NamedArrays

nodes = [:S, :K, :P, :alpha, :L, :B, :O, :beta, :C, :M, :R]
producers = [:K, :L, :C, :alpha, :beta]
consumers = [:S, :K, :P, :L, :B, :O, :C, :M, :R]

edges = [(:K, :P), (:S, :P), (:P, :L), (:P, :alpha), 
    (:alpha, :B), (:alpha, :L), (:alpha, :R), (:L, :B), (:B, :O),
    (:O, :beta), (:beta, :R), (:beta, :C), (:beta, :M)]
edges = [edges; [(j, i) for (i, j) in edges]]

dist_vec = [1, 2, 6, 3, 4, 5, 7, 5, 3, 4, 2, 2, 3]*100
distance = Dict(zip(edges, [dist_vec; dist_vec]))

days = [:Weds, :Thurs, :Fri]

loss = 0.0001

matrix = zeros(length(consumers), length(days))
demand = NamedArray(matrix, (consumers, days), ("Consumer","Day"))
demand[:S, :Weds]=40.8; demand[:S, :Thurs]=100; demand[:S, :Fri]=50.5;
demand[:P, :Weds]=28.8; demand[:P, :Thurs]=30.3; demand[:P, :Fri]=28.8;
demand[:K, :Weds]=27.3; demand[:K, :Thurs]=27.3; demand[:K, :Fri]=27.3;
demand[:L, :Weds]=30.1; demand[:L, :Thurs]=28.8; demand[:L, :Fri]=27.3;
demand[:C, :Weds]=30.0; demand[:C, :Thurs]=25.5; demand[:C, :Fri]=31.2;
demand[:B, :Weds]=30.1; demand[:B, :Thurs]=31.5; demand[:B, :Fri]=30.1;
demand[:O, :Weds]=13.7; demand[:O, :Thurs]=8.3; demand[:O, :Fri]=16.4;
demand[:M, :Weds]=27.3; demand[:M, :Thurs]=21.9; demand[:M, :Fri]=24.6;
demand[:R, :Weds]=19.2; demand[:R, :Thurs]=20.5; demand[:R, :Fri]=20.5;

matrix = zeros(length(producers), length(days))
supply = NamedArray(matrix, (producers, days), ("Producer","Day"))
supply[:alpha, :Weds] = 1500; supply[:alpha, :Thurs] = 300; supply[:alpha, :Fri] = 7000;
supply[:beta, :Weds] = 2500; supply[:beta, :Thurs] = 200; supply[:beta, :Fri] = 5000;
supply[:L, :Weds] = 1500; supply[:L, :Thurs] = 2000; supply[:L, :Fri] = 1500;
supply[:K, :Weds] = 2000; supply[:K, :Thurs] = 1500; supply[:K, :Fri] = 1000;
supply[:C, :Weds] = 1500; supply[:C, :Thurs] = 2300; supply[:C, :Fri] = 1800;

matrix = zeros(length(nodes), length(days))
b = NamedArray(matrix, (nodes, days), ("Nodes", "Day"))
for n in nodes
    for t in days
        if !(n in producers)
            b[n, t] = -demand[n, t]
        elseif !(n in consumers)
            b[n, t] = supply[n, t]
        else
            b[n, t] = supply[n, t] - demand[n, t]
        end
    end
end

In [75]:
function P(x, i, j)
    return (1-loss*distance[(i,j)])*x
end

P (generic function with 1 method)

In [105]:
# Model
using Clp, JuMP

m = Model(solver=ClpSolver())

@variable(m, x[edges, days] >= 0) # energy flow from node i to j

# # useful expressions
@expression(m, flow_in[k in nodes, t in days], 
    sum((1-loss*distance[(i,j)])*x[(i,j), t] for (i,j) in edges if j == k))
@expression(m, flow_out[k in nodes, t in days], 
    sum(x[(i,j), t] for (i,j) in edges if i == k))
@expression(m, energy_lost, sum(loss*distance[a]*x[a, t] for a in edges for t in days))
                                
@constraint(m, power_balance[k in nodes, t in days], flow_out[k, t] - flow_in[k, t] <= b[k, t])
# @constraint(m, energy_constr[t in days], sum(x[a, t] for a in edges) <= sum(supply[p, t] for p in producers))
                                                
@objective(m, Min, energy_lost)
                                                
status = solve(m)

println("------Solution------")
println("Gift Matrix: ", matrix)
println(getvalue(x))
println("Total energy losses will be ", getobjectivevalue(m), " kWh.")

------Solution------
Gift Matrix: [-40.8 -100.0 -50.5; 1972.7 1472.7 972.7; -28.8 -30.3 -28.8; 1500.0 300.0 7000.0; 1469.9 1971.2 1472.7; -30.1 -31.5 -30.1; -13.7 -8.3 -16.4; 2500.0 200.0 5000.0; 1470.0 2274.5 1768.8; -27.3 -21.9 -24.6; -19.2 -20.5 -20.5]
x: 2 dimensions:
[    (:K, :P),:]
  [    (:K, :P), Weds] = 71.14409400123685
  [    (:K, :P),Thurs] = 133.67759224902082
  [    (:K, :P),  Fri] = 81.142032570604
[    (:S, :P),:]
  [    (:S, :P), Weds] = 0.0
  [    (:S, :P),Thurs] = 0.0
  [    (:S, :P),  Fri] = 0.0
[    (:P, :L),:]
  [    (:P, :L), Weds] = 0.0
  [    (:P, :L),Thurs] = 0.0
  [    (:P, :L),  Fri] = 0.0
[(:P, :alpha),:]
  [(:P, :alpha), Weds] = 0.0
  [(:P, :alpha),Thurs] = 0.0
  [(:P, :alpha),  Fri] = 0.0
[(:alpha, :B),:]
  [(:alpha, :B), Weds] = 31.35416666666667
  [(:alpha, :B),Thurs] = 32.8125
  [(:alpha, :B),  Fri] = 31.35416666666667
[(:alpha, :L),:]
  [(:alpha, :L), Weds] = 0.0
  [(:alpha, :L),Thurs] = 0.0
  [(:alpha, :L),  Fri] = 0.0
[(:alpha, :R),:]
  [(:alpha, :

In [95]:
total_supply = Dict(zip(days, sum(supply, 1)))

Dict{Symbol,Float64} with 3 entries:
  :Fri   => 16300.0
  :Weds  => 9000.0
  :Thurs => 6300.0

In [102]:
total_supply = (getvalue(flow_out[:alpha, :Weds] + flow_out[:beta, :Weds] 
    + flow_out[:L, :Weds] + flow_out[:K, :Weds] + flow_out[:C, :Weds])
    + demand[:L, :Weds] + demand[:K, :Weds] + demand[:C, :Weds])






251.90526063283795

In [81]:
b

11×3 Named Array{Float64,2}
Nodes ╲ Day │  :Weds  :Thurs    :Fri
────────────┼───────────────────────
:S          │  -40.8  -100.0   -50.5
:K          │ 1972.7  1472.7   972.7
:P          │  -28.8   -30.3   -28.8
:alpha      │ 1500.0   300.0  7000.0
:L          │ 1469.9  1971.2  1472.7
:B          │  -30.1   -31.5   -30.1
:O          │  -13.7    -8.3   -16.4
:beta       │ 2500.0   200.0  5000.0
:C          │ 1470.0  2274.5  1768.8
:M          │  -27.3   -21.9   -24.6
:R          │  -19.2   -20.5   -20.5

In [96]:
total_demand = Dict(zip(days, sum(demand, 1)))

Dict{Symbol,Float64} with 3 entries:
  :Fri   => 256.7
  :Weds  => 247.3
  :Thurs => 294.1

In [90]:
net = Dict(zip(days, total_supply[t] - total_demand[t] for t in days))

Dict{Symbol,Any} with 3 entries:
  :Fri   => 16043.3
  :Weds  => 8752.7
  :Thurs => 6005.9

In [92]:
getvalue(sum(x[a, :Weds] for a in edges))

425446.58346557873